In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Install Prerequisite libraries

In [2]:
pip install transformers

     |████████████████████████████████| 4.2 MB 4.9 MB/s 
     |████████████████████████████████| 6.6 MB 48.0 MB/s 
     |████████████████████████████████| 596 kB 49.3 MB/s 
     |████████████████████████████████| 84 kB 2.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
pip install underthesea

     |████████████████████████████████| 7.6 MB 2.9 MB/s 
     |████████████████████████████████| 965 kB 58.5 MB/s 
     |████████████████████████████████| 235 kB 56.5 MB/s 
     |████████████████████████████████| 581 kB 48.4 MB/s 


Check GPU

In [4]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [5]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

#Load libraries

In [1]:
from os.path import join
import os
import pandas as pd
from transformers import AutoTokenizer,AutoConfig,AutoModelForSequenceClassification,get_linear_schedule_with_warmup
import re
from underthesea import sent_tokenize,word_tokenize,pos_tag
import string
from torch.utils.data import DataLoader, Dataset
import torch
from torch import nn
from tqdm.auto import tqdm
import numpy as np
from sklearn.metrics import classification_report,confusion_matrix,f1_score,precision_score,recall_score,accuracy_score 
from glob import glob
from collections import Counter
import random

# Global variables

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [3]:
# folder name
folder="vietnamese_students_feedback"
raw_dataset="raw_dataset"
models="models"
name='xlm-roberta-large'
#approach='entailment'
max_length=256
preproccesd_data="preproccesd_data"

# directories
root_path="/content/drive/MyDrive"
folder_path=join(root_path,folder)
raw_dataset_path=join(root_path,folder,raw_dataset)
models_path=join(root_path,folder,models)
model_path=f'{models_path}/{name}-{str(max_length)}/'
preproccesd_data_path=f'{folder_path}/{preproccesd_data}'
approach_path=f'{model_path}'

Create directories

In [4]:
os.makedirs(model_path, exist_ok=True)
os.makedirs(raw_dataset_path, exist_ok=True)
os.makedirs(preproccesd_data_path, exist_ok=True)

Because the dataset contains english and vietnamese -> use bert multilingual

In [5]:
tokenizer = AutoTokenizer.from_pretrained(name)
# cased_tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

#Preprocessing

In [12]:
train=pd.read_csv(join(raw_dataset_path,'train.csv'))
valid=pd.read_csv(join(raw_dataset_path,'valid.csv'))

In [13]:
train.head()

,sentence,sentiment,topic
0,slide giáo trình đầy đủ .,2,1
1,"nhiệt tình giảng dạy , gần gũi với sinh viên .",2,0
2,đi học đầy đủ full điểm chuyên cần .,0,1
3,chưa áp dụng công nghệ thông tin và các thiết ...,0,0
4,"thầy giảng bài hay , có nhiều bài tập ví dụ ng...",2,0


Only take topic column as label for task topic classification and sentence column as input

The dataset has some specific token such as
- doubledot -> :
- dot -> .
- colonsmilesmile -> :))
- vdotv -> v.v
- pronoun -> wzjwz261
- fraction -> /
- cnpm -> công nghệ phần mềm
- x -> remove x



-> remove them

convert multiple x characters -> one x character

In [6]:
punctuations=list(string.punctuation)

In [7]:
def tokenize(s,join=False): 
    sentences = sent_tokenize(s) 
    # tokens=[]
    _sentences=[]
    for sentence in sentences:
      # tokens+=sentence
      tokens=[]
      for token in word_tokenize(sentence,):
        token=token.replace("-", " ")
        if join==True:
          token=token.replace(" ","_")
        
        tokens.append(token)
      _sentences.append(" ".join(tokens))
    return " ".join(_sentences)

In [8]:
def normalizer(text,joinf=False):
  def remove_symbols(s):
    return re.sub(r'\b(colon\w+)\b'," ",s)

  def remove_fraction(s):
    return re.sub(r'\b(fraction)\b','/',s)

  def replace_dot(s):
    return s.replace('dot','.')
    # return re.sub(r'\b(dot)\b',".",s)

  def replace_doubledot(s):
    return s.replace('doubledot',':')
    
  def uncased(s):
    return s.lower()

  def fix_whitespaces(s):
    return re.sub(r'\s+',' ',s)

  def remove_entities(s):
    return re.sub(r'\b((\w+|)wzjwz\d+)\b'," ",s)

  def remove_vv(s):
    return s.replace('v.v'," ")

  def remove_punctuations(s):
      return ''.join(ch for ch in s if ch not in punctuations)
  def norm_major(s):
      majors={
          'cnpm':"công nghệ phần mềm",
          'khmt':"khoa học máy tính",
          'ktmt':"kĩ thuật máy tính",
          'cntt':"công nghệ thông tin",
          'av1':"anh văn 1",
          'av2':"anh văn 2",
          'av3':"anh văn 3"
      }
      for k,v in majors.items():
        s=re.sub(r'\b({})\b'.format(k),v,s)
      return s
  def remove_ps(s):
    return s.replace("p / s"," ")
  return fix_whitespaces(uncased(remove_punctuations(tokenize(norm_major(remove_ps(remove_fraction(remove_vv(remove_entities(remove_symbols(replace_dot(replace_doubledot(text)))))))),join)))).strip()

In [9]:
# def normalizer(text,join=False):
#   def remove_symbols(s):
#     return re.sub(r'\b(colon\w+)\b'," ",s)

#   def remove_fractions(s):
#     return re.sub(r'\b(fraction)\b'," ",s)

#   # def norm_year(s):
#   #   for year in re.findall(r'\b(([0-9])(k|K)([0-9]))\b',s):
#   #     s=re.sub(r'\b({})\b'.format(year[0]),f'{year[1]}00{year[-1]}',s)
#   #   return s

#   def norm_major(s):
#     majors={
#         'cnpm':"công nghệ phần mềm",
#         'khmt':"khoa học máy tính",
#         'ktmt':"kĩ thuật máy tính",
#         'cntt':"công nghệ thông tin",
#         'av1':"anh văn 1",
#         'av2':"anh văn 2",
#         'av3':"anh văn 3"
#     }
#     for k,v in majors.items():
#       s=re.sub(r'\b({})\b'.format(k),v,s)
#     return s
  
#   def remove_dup_x(s):
#     return re.sub(r'\b((x|X)+)\b',' ',s)

#   def replace_dot(s):
#     return s.replace('dot','.')
#     # return re.sub(r'\b(dot)\b',".",s)

#   def replace_doubledot(s):
#     return s.replace('doubledot',':')
    
#   def uncased(s):
#     return s.lower()

#   def fix_whitespaces(s):
#     return re.sub(r'\s+',' ',s)

#   def remove_entities(s):
#     return re.sub(r'\b((\w+|)wzjwz\d+)\b'," ",s)

#   def remove_vv(s):
#     return s.replace('v.v'," ")

#   def remove_punctuations(s):
#       return ''.join(ch for ch in s if ch not in punctuations)

#   return fix_whitespaces(norm_major(uncased(tokenize(remove_fractions(remove_dup_x(remove_punctuations(remove_vv(remove_entities(remove_symbols(replace_dot(replace_doubledot(text)))))))),join)))).strip()

In [10]:
# s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
# s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'
# def remove_accents(input_str):
# 	s = ''
# 	for c in input_str:
# 		if c in s1:
# 			s += s0[s1.index(c)]
# 		else:
# 			s += c
# 	return s

In [11]:
def multiple_normalizer(samples,join=False):
  bar=tqdm(samples)
  _samples=[]
  for sample in samples:
    _sample=normalizer(sample,join)
    _samples.append(_sample)
    bar.update()
  return _samples

In [25]:
# train['join_token']=multiple_normalizer(train['sentence'].tolist(),True)
train['cleaned_sentence']=multiple_normalizer(train['sentence'].tolist(),False)

  0%|          | 0/11426 [00:00<?, ?it/s]

In [26]:
valid['cleaned_sentence']=multiple_normalizer(valid['sentence'].tolist(),False)

  0%|          | 0/1583 [00:00<?, ?it/s]

In [27]:
# train['no_accents_sentence']=train['accents_sentence'].map(remove_accents)
# valid['no_accents_sentence']=valid['accents_sentence'].map(remove_accents)

In [28]:
def seperate_sentence(s):
  return tokenizer.tokenize(s)
  # return s.split()

In [29]:
def get_seq_len(s):
  tokens=seperate_sentence(s)
  return len(tokens)

In [30]:
train['seq_len']=train['cleaned_sentence'].map(get_seq_len)
valid['seq_len']=valid['cleaned_sentence'].map(get_seq_len)

In [31]:
train['seq_len'].unique()

array([  5,   9,   8,  17,  13,  25,  16,  12,  29,  15,  10,   4,  28,
        26,  39,  11,   6,  34,  23,  31,  22,  24,  38,   7,  14,  30,
        32,  19,   3,  20,  37,  36,  41,   2,  18,  21,   1,  33,  42,
        51,  44,  27,  53,  45,  48,  71,  62,  59,  58,  35,  40,  43,
        77,  60,  54,  65,  81,  49,  89,  96,  50,  57,  98,  72,  46,
        52, 155,  63,  84,  47,  73,  64,  66,  67,  95,  55,  74, 104,
        61,  70,  83,  69,  68, 114,  56])

In [32]:
valid['seq_len'].unique()

array([  5,   3,   6,  33,  23,  17,   9,  13,  15,   8,  10,   7,  48,
         4,  14,  19,  46,  11,  40,  24,  25,  29,   2,  30,  12,  31,
        20,  26,  85,  53,  16,  18,  21,  22,  27,  28,  39,  54,  34,
        37,   1,  88,  32,  35,  38,  41,  43,  63,  49,  36,  42,  52,
       154,  45,  57,  44])


-> assign max_length is 256

In [33]:
train=train.sample(frac=1)

In [34]:
train.to_csv(join(preproccesd_data_path,'train.csv'),index=False)
valid.to_csv(join(preproccesd_data_path,'valid.csv'),index=False)

#Feature extraction

In [6]:
train=pd.read_csv(join(preproccesd_data_path,'train.csv'))
valid=pd.read_csv(join(preproccesd_data_path,'valid.csv'))

In [7]:
train[train['cleaned_sentence'].isnull()]

,sentence,sentiment,topic,cleaned_sentence,seq_len


In [8]:
valid[valid['cleaned_sentence'].isnull()]

,sentence,sentiment,topic,cleaned_sentence,seq_len


In [9]:
class ClassificationDataset(Dataset):
    def __init__(self, input_ids, attention_mask,labels,device,token_type_ids=None):
        self.labels = labels
        self.input_ids=input_ids
        self.attention_mask=attention_mask
        self.token_type_ids=token_type_ids
        self.device=device
        
    def __len__(self):
            return len(self.labels)

    def __getitem__(self, idx):
        label = self.labels[idx]
        input_ids = self.input_ids[idx]
        attention_mask = self.attention_mask[idx]
        sample = {"inputs":{"input_ids": input_ids.to(self.device),'attention_mask':attention_mask.to(self.device)}, "labels": label.to(self.device)}
        if self.token_type_ids!=None:
          sample['inputs'].update({'token_type_ids':self.token_type_ids[idx].to(self.device)})
        return sample

In [10]:
train_batch_size=8
num_training_samples=len(train)
num_training_batches=int((num_training_samples/train_batch_size)+0.5)

In [11]:
valid_batch_size=8
num_valid_samples=len(valid)
num_valid_batches=int((num_valid_samples/valid_batch_size)+0.5)

In [12]:
return_tensors='pt'

In [13]:
labels=train['topic'].unique().tolist()
labels

[1, 0, 3, 2]

In [14]:
def create_dataloader(tokenizer,samples,labels,max_length,batch_size,padding=True,truncation=True,return_tensors='pt',device='cpu',drop_last=False):
  x=tokenizer(samples ,padding=padding,truncation=truncation,max_length=max_length,return_tensors=return_tensors)
  y=torch.Tensor(labels).to(torch.long)
  dataset={i:x[i] for i in x}
  dataset.update({"labels":y})
  dataset=ClassificationDataset(**dataset,device=device)
  dataset=DataLoader(dataset,batch_size=batch_size,drop_last=drop_last,shuffle=True)
  return dataset

In [15]:
len(train)

11426

In [16]:
# uncased_train_dataloader=create_dataloader(tokenizer,train['no_accents_sentence'].tolist()+train['accents_sentence'].tolist(),train['topic'].tolist(),max_length=max_length,return_tensors=return_tensors,device=device,drop_last=False,batch_size=train_batch_size)
train_dataloader=create_dataloader(tokenizer,train['cleaned_sentence'].tolist(),train['topic'].tolist(),max_length=max_length,return_tensors=return_tensors,device=device,drop_last=False,batch_size=train_batch_size)

In [17]:
for i in train_dataloader:
  print(i['inputs']['input_ids'].shape)
  break

torch.Size([8, 157])


In [18]:
valid_dataloader=create_dataloader(tokenizer,valid['cleaned_sentence'].tolist(),valid['topic'].tolist(),max_length=max_length,return_tensors=return_tensors,device=device,drop_last=False,batch_size=valid_batch_size)

In [19]:
for i in valid_dataloader:
  print(i['inputs']['input_ids'].shape)
  break

torch.Size([8, 156])


#Trainer

In [20]:
epochs=30
num_training_steps=epochs * len(train_dataloader)
num_warmup_steps=500
init_lr=1e-5
eps =1e-8

In [21]:
num_training_steps,num_warmup_steps

(42870, 500)

In [22]:
num_labels=len(labels)

In [23]:
classifier_dropout=0.1

In [24]:
# from transformers import BertPreTrainedModel
# from transformers import AutoModel,AutoConfig
# from torch import nn
# class mBERTForClassification(BertPreTrainedModel):
#   def __init__(self, config,from_pretrained=False):
#     super(mBERTForClassification,self).__init__(config)
#     self.config = config
#     self.bert_name=self.config._name_or_path
#     self.from_pretrained=from_pretrained

#     if self.from_pretrained:
#       self.bert= AutoModel.from_pretrained(self.bert_name,config=self.config)
#     else:
#       self.bert= AutoModel.from_config(config=self.config)

#     self.pooled_output1=nn.Linear(self.config.hidden_size,self.config.hidden_size)
    
#     self.dropout1 = nn.Dropout(self.config.classifier_dropout)


#     self.classifier = nn.Linear(self.config.hidden_size, self.config.num_labels)

#     # Initialize weights and apply final processing
#     self.post_init()

#   def forward(self,inputs):
#     outputs = self.bert(**inputs,output_hidden_states=True)
#     last_layers=torch.sum(torch.stack(outputs.hidden_states[-4:]), dim=0)

#     pooled_output=self.pooled_output1(last_layers[:,0,:])
#     pooled_output=torch.tanh(pooled_output)
#     pooled_output = self.dropout1(pooled_output)

#     logits = self.classifier(pooled_output)
#     return logits

In [25]:
config=AutoConfig.from_pretrained(name)
config.classifier_dropout=classifier_dropout
config.num_labels=num_labels
config

XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-large",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": 0.1,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.19.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

In [26]:
config.save_pretrained(approach_path)

In [27]:
# model=mBERTForClassification(config).to(device)
model=AutoModelForSequenceClassification.from_pretrained(name,config=config).to(device)

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.out

In [28]:
# loss_fn = nn.CrossEntropyLoss().to(device)

In [29]:
optimizer=torch.optim.AdamW(model.parameters(),lr=init_lr,weight_decay =0.01,eps=eps)

In [30]:
approach_path

'/content/drive/MyDrive/vietnamese_students_feedback/models/xlm-roberta-large-256/'

In [31]:
lr_scheduler=get_linear_schedule_with_warmup(optimizer,num_warmup_steps=num_warmup_steps,num_training_steps=num_training_steps,)

In [32]:
def save_model(epoch,model,optimizer,training_loss,valid_loss,path,step=None):
  torch.save({
          'epoch': epoch,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'training_loss': training_loss,
          'valid_loss': valid_loss,
          'step':step
          }, join(path,f'ckpt{epoch}.pt'))

Train per epoch

In [33]:
def run_train(optimizer,dataloader,lr_scheduler):
  model.train(True)
  loop = tqdm(dataloader)
  loss=0
  for batch in dataloader:
    # model.zero_grad(set_to_none=True)
    optimizer.zero_grad()
    inputs=batch['inputs']
    inputs.update({"labels":batch['labels']})
    outputs=model(**inputs)
    # _loss = loss_fn(logits.view(-1, num_labels), batch['labels'].view(-1))
    outputs.loss.backward()

    loss+=outputs.loss.item()
    optimizer.step()
    lr_scheduler.step()
    loop.set_postfix({f'train_loss_per_batch':outputs.loss.item()})
    loop.update()      
    
  return loss/len(dataloader)

In [34]:
def run_valid(dataloader):
  model.eval()
  loss=0
  with torch.no_grad():
    for batch in dataloader:
      inputs=batch['inputs']
      inputs.update({"labels":batch['labels']})
      outputs=model(**inputs)

      # _loss = loss_fn(logits.view(-1, num_labels), batch['labels'].view(-1))

      loss+=outputs.loss.item()

  return loss/len(dataloader)

In [35]:
approach_path

'/content/drive/MyDrive/vietnamese_students_feedback/models/xlm-roberta-large-256/'

In [ ]:
train_loss_per_step=[]
valid_loss_per_step=[]
min_loss=np.inf
count_stopping=0
for epoch in range(epochs):
  train_loss=run_train(optimizer,train_dataloader,lr_scheduler)
  valid_loss=run_valid(valid_dataloader)

  train_loss_per_step.append(train_loss)
  valid_loss_per_step.append(valid_loss)
  
  print(f'epoch: {epoch} train_loss: {train_loss} valid_loss: {valid_loss}')
  if valid_loss<min_loss:
    min_loss=valid_loss
    save_model(epoch,model,optimizer,train_loss,valid_loss,f'{approach_path}')
    count_stopping=0
  else:
    count_stopping+=1
  
  if count_stopping>5:
    break


  0%|          | 0/1429 [00:00<?, ?it/s]

epoch: 0 train_loss: 0.5081220485786194 valid_loss: 0.3860593673848368


  0%|          | 0/1429 [00:00<?, ?it/s]

epoch: 1 train_loss: 0.317129366994986 valid_loss: 0.3297661553216966


  0%|          | 0/1429 [00:00<?, ?it/s]

...

  0%|          | 0/1429 [00:00<?, ?it/s]

epoch: 4 train_loss: 0.1953788520155251 valid_loss: 0.3638986541661481


  0%|          | 0/1429 [00:00<?, ?it/s]

epoch: 5 train_loss: 0.15918787812554727 valid_loss: 0.3885211709638672


  0%|          | 0/1429 [00:00<?, ?it/s]

# Inference

In [12]:
import pandas as pd
test=pd.read_csv(f'{raw_dataset_path}/test.csv')
test  

,sentence,sentiment,topic
0,nói tiếng anh lưu loát .,2,0
1,giáo viên rất vui tính .,2,0
2,cô max có tâm .,2,0
3,"giảng bài thu hút , dí dỏm .",2,0
4,"giáo viên không giảng dạy kiến thức , hướng dẫ...",0,0
...,...,...,...
3161,"các slide khó hiểu , ngôn ngữ trong slide phức...",0,0
3162,giáo viên giảng dạy có tâm huyết .,2,0
3163,chia sẻ cho em nhiều điều hay .,2,0
3164,em tiếp thu chậm .,0,0


In [13]:
def metrics_evaluate(y_true,y_pred):
  acc=accuracy_score(y_true,y_pred)

  micro_precision=precision_score(y_true,y_pred,average='micro')
  micro_recall=recall_score(y_true,y_pred,average='micro')
  micro_f1=f1_score(y_true,y_pred,average='micro')

  macro_precision=precision_score(y_true,y_pred,average='macro')
  macro_recall=recall_score(y_true,y_pred,average='macro')
  macro_f1=f1_score(y_true,y_pred,average='macro')

  average_precision=precision_score(y_true,y_pred,average='weighted')
  average_recall=recall_score(y_true,y_pred,average='weighted')
  average_f1=f1_score(y_true,y_pred,average='weighted')

  return {
      'acc':acc,
      'micro_precision':micro_precision,
      'micro_recall':micro_recall,
      'micro_f1':micro_f1,

      'macro_precision':macro_precision,
      'macro_recall':macro_recall,
      'macro_f1':macro_f1,

      'average_precision':average_precision,
      'average_recall':average_recall,
      'average_f1':average_f1
  }



In [14]:
samples=test['sentence'].tolist()

In [15]:
def load_best_checkpoint(model_directory):
  min_loss=np.inf
  best_ckpt=""
  ckpt_files=glob(f'{model_directory}/*.pt')
  bar=tqdm(ckpt_files)
  for f in ckpt_files:
    ckpt=torch.load(f,map_location=device)
    if min_loss>ckpt['valid_loss']:
      min_loss=ckpt['valid_loss']
      best_ckpt=f 
      bar.update()
  return best_ckpt,min_loss

In [ ]:
ckpt_path,valid_loss=load_best_checkpoint(approach_path)

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
ckpt_path

In [16]:
config = AutoConfig.from_pretrained(f'{approach_path}/config.json')
config._name_or_path=name

In [17]:
# model=mBERTForClassification(config).to(device)

model=AutoModelForSequenceClassification.from_pretrained(name,config=config).to(device)

Downloading:   0%|          | 0.00/2.09G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.den

In [18]:
checkpoint = torch.load("/content/drive/MyDrive/vietnamese_students_feedback/models/xlm-roberta-large-256/ckpt2.pt",map_location=device)

In [19]:
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [20]:
y_true=test['topic'].tolist()

In [21]:
def predict(inputs,max_length,return_tensors='pt',device='cpu',normalizer=None):
  #inputs: list of texts
  #normalizer: function -> clean dataset and return a new input with the same format
  model.eval()
  y_pred=[]
  with torch.no_grad():
    loop = tqdm(range(len(inputs)))
    for idx,sample in enumerate(inputs):
      if normalizer!=None:
        sample=normalizer(sample)
      _inputs=tokenizer(sample,padding=True,truncation=True,max_length=max_length,return_tensors=return_tensors).to(device)
      outputs=model(**_inputs)
      # logits=cased_logits+uncased_logits
      # logits=uncased_logits
      probs=torch.nn.functional.softmax(outputs.logits,dim=1)
      preds=torch.argmax(probs,dim=1)
      y_pred.append(preds)
      loop.update()
  return torch.Tensor(y_pred)

In [22]:
y_pred=predict(test['sentence'],max_length,device=device,normalizer=normalizer)

  0%|          | 0/3166 [00:00<?, ?it/s]

In [23]:
y_pred

tensor([0., 0., 0.,  ..., 0., 3., 1.])

In [24]:
#y_pred=torch.argmax(y_pred,dim=1)

In [25]:
y_pred=y_pred.tolist()

In [26]:
test['pred']=y_pred

In [27]:
test.to_csv("outputs.csv",index=False)

In [28]:
metrics=metrics_evaluate(y_true,y_pred)
metrics

{'acc': 0.8954516740366393,
 'average_f1': 0.892052110799448,
 'average_precision': 0.8923371310558289,
 'average_recall': 0.8954516740366393,
 'macro_f1': 0.7948634470285963,
 'macro_precision': 0.8248546735007211,
 'macro_recall': 0.7823364540497673,
 'micro_f1': 0.8954516740366393,
 'micro_precision': 0.8954516740366393,
 'micro_recall': 0.8954516740366393}

In [29]:
pd.DataFrame(metrics,index=range(0,1))

,acc,micro_precision,micro_recall,micro_f1,macro_precision,macro_recall,macro_f1,average_precision,average_recall,average_f1
0,0.895452,0.895452,0.895452,0.895452,0.824855,0.782336,0.794863,0.892337,0.895452,0.892052


In [30]:
confusion_matrix(y_true,y_pred)

array([[2171,   97,    4,   18],
       [  96,  458,    6,   12],
       [   2,    3,  140,    0],
       [  44,   46,    3,   66]])

In [31]:
print(classification_report(y_true,y_pred,digits=5))

              precision    recall  f1-score   support

           0    0.93861   0.94803   0.94330      2290
           1    0.75828   0.80070   0.77891       572
           2    0.91503   0.96552   0.93960       145
           3    0.68750   0.41509   0.51765       159

    accuracy                        0.89545      3166
   macro avg    0.82485   0.78234   0.79486      3166
weighted avg    0.89234   0.89545   0.89205      3166



In [32]:
def find_wrong_samples(y_true,y_pred):
  y_true=torch.Tensor(y_true)
  y_pred=torch.Tensor(y_pred)
  return torch.where(y_true!=y_pred)[0].tolist()

In [33]:
wrong_samples=find_wrong_samples(y_true,y_pred)

In [34]:
wrong_samples=test.iloc[wrong_samples]

In [35]:
wrong_samples

,sentence,sentiment,topic,pred
12,trong trường macbook thầy số hai thì không có ...,2,3,0.0
23,nên đưa ra một vài phương pháp học lập trình h...,0,0,1.0
43,"cô vui tính , học không áp lực .",2,1,0.0
48,"cô nhiệt tình , giảng bài hiệu quả .",2,1,0.0
63,rất thực tế không dài dòng .,2,3,1.0
...,...,...,...,...
3150,có một ít áp lực .,1,0,3.0
3158,ghi lý thuyết và bài tập trên bảng nhiều quá t...,0,0,1.0
3159,môn thiết kế mạng cần cho sinh viên thực hành ...,0,1,2.0
3161,"các slide khó hiểu , ngôn ngữ trong slide phức...",0,0,1.0


In [36]:
wrong_samples[wrong_samples['topic']==0]

,sentence,sentiment,topic,pred
23,nên đưa ra một vài phương pháp học lập trình h...,0,0,1.0
84,em không thích như vậy tí xíu nào .,0,0,3.0
89,không slide .,0,0,1.0
102,đồ án phần bắt buộc chỉ chiếm 1dot5 điểm và ch...,0,0,1.0
110,kiến thức chuyên môn sâu .,2,0,1.0
...,...,...,...,...
3136,em thấy vấn đề điểm danh khi đi học là không h...,0,0,1.0
3150,có một ít áp lực .,1,0,3.0
3158,ghi lý thuyết và bài tập trên bảng nhiều quá t...,0,0,1.0
3161,"các slide khó hiểu , ngôn ngữ trong slide phức...",0,0,1.0


In [37]:
wrong_samples[wrong_samples['topic']==1]

,sentence,sentiment,topic,pred
43,"cô vui tính , học không áp lực .",2,1,0.0
48,"cô nhiệt tình , giảng bài hiệu quả .",2,1,0.0
86,thời gian các bạn lên làm bài tập thì thầy nên...,0,1,0.0
121,"ví dụ phù hợp với nội dung kiến thức , hướng d...",1,1,0.0
171,sử dụng quá nhiều tiếng anh để giảng bài trong...,0,1,0.0
...,...,...,...,...
3022,lớp học sôi động .,2,1,0.0
3023,đến lớp đúng giờ !,2,1,0.0
3140,"tạo ra một môi trường năng động , giúp đỡ nhiề...",2,1,0.0
3147,cho sinh viên thực hành một đề bài quản lý thô...,0,1,0.0


In [38]:
wrong_samples[wrong_samples['topic']==2]

,sentence,sentiment,topic,pred
125,có nhiều thầy cô đến lớp không đúng giờ .,0,2,0.0
638,để sinh viên thực hành nhiều hơn .,0,2,1.0
866,đề thi của thầy khá hay khi khai thác thực tiễn .,2,2,1.0
871,sẽ có thông báo đi học cụ thể trên phòng máy đ...,2,2,1.0
1055,sử dụng thời gian ở lớp hiệu quả .,2,2,0.0


In [39]:
wrong_samples[wrong_samples['topic']==3]

,sentence,sentiment,topic,pred
12,trong trường macbook thầy số hai thì không có ...,2,3,0.0
63,rất thực tế không dài dòng .,2,3,1.0
95,học thì quá ít nhưng khi thi thì quá nhiều yêu...,0,3,1.0
115,sự nhiệt tình tận tâm .,2,3,0.0
126,đôi lúc sinh viên theo không kịp .,0,3,0.0
...,...,...,...,...
3073,sử dụng nhiều ví dụ giúp sinh viên nắm kiến th...,2,3,0.0
3095,ngoài thời gian cố định như trong thời khóa bi...,1,3,1.0
3096,nên trả lời những câu hỏi của sinh viên một cá...,0,3,0.0
3107,hầu như chúng em không tiếp thu được gì .,0,3,1.0
